In [57]:
import pandas as pd
import numpy as np

In [58]:
Arbo_V = 'Dengue'
Ano = ['2021', '2020']

In [59]:
df = pd.read_pickle("../Datasets/dengue_ano.pkl")
censo = pd.read_pickle("../Datasets/ibge_2010.pkl")
if Arbo_V == 'Dengue':
    obitos = pd.read_pickle("../Datasets/dengue_obitos.pkl")

In [62]:
df = df.merge(censo, left_on='Município', right_on='Município')
df['Casos/Hab 100M'] = (df[f'{Ano[0]}']/df["População 2010"]*100000)
df['Casos/Hab 100M'] = df['Casos/Hab 100M'].astype('float32').round(2)

In [ ]:
if Arbo_V == "Dengue":
    df = df.merge(obitos, left_on='Município', right_on='Município')
    df['Mort/Hab 100M'] = (df[f'{Ano[0]}']/df["Obitos"]*100000)
    df['Mort/Hab 100M'] = df['Mort/Hab 100M'].astype('float32').round(2)

In [64]:
df[f'Tx Crescimento dos anos {Ano[0]} e {Ano[1]}'] = np.around((df[Ano[0]].astype('float32') - df[Ano[1]].astype('float32')).divide(df[Ano[1]].astype('float32')) * 100)

In [66]:
if Arbo_V == "Dengue":
    aux = df.nlargest(10, 'Casos/Hab 100M')[['Código', 'Município', 'População 2010', f'{Ano[0]}', 'Obitos', 'Casos/Hab 100M', 'Mort/Hab 100M', f'Tx Crescimento dos anos {Ano[0]} e {Ano[1]}']]
else:
    aux = df.nlargest(10, 'Casos/Hab 100M')[['Código', 'Município', 'População 2010', f'{Ano[0]}', 'Casos/Hab 100M', f'Tx Crescimento dos anos {Ano[0]} e {Ano[1]}']]

In [71]:
total_casos = pd.DataFrame(df.iloc[:, 2:-3].sum(), df.columns[2:-3], columns=['Casos'])

In [85]:
total_casos.Casos = total_casos.Casos.astype('int32')

In [86]:
total_casos.describe()

,Casos,População 2010
count,11.000000,11.0
mean,60495.727273,14019055.0
std,32448.298276,0.0
min,14222.000000,14019055.0
25%,31123.500000,14019055.0
50%,74543.000000,14019055.0
75%,84360.500000,14019055.0
max,107509.000000,14019055.0


In [88]:
taxa_20_19 = round((total_casos.loc["2020"][0] - total_casos.loc["2019"][0]) / total_casos.loc["2019"][0] * 100, 1)
taxa_20_21 = round((total_casos.loc["2020"][0] - total_casos.loc["2021"][0]) / total_casos.loc["2021"][0] * 100, 1)

str(f'Taxa de crescimento de {Arbo_V} dos anos de 2019 e 2021 comparados com 2020 foi de {taxa_20_19}% e {taxa_20_21}%')

'Taxa de crescimento de Dengue dos anos de 2019 e 2021 comparados com 2020 foi de 26.6% e 187.6%'

In [89]:
total_casos["População 2010"] = df["População 2010"].sum()
total_casos["Casos/Hab 100M"] = round(total_casos.iloc[:, 0]/total_casos.loc[:, "População 2010"] * 100000, 1)

In [ ]:
if Arbo_V == 'Dengue':
    total_casos["Obitos"] = list(obitos.iloc[:, 3:14].sum())

In [ ]:
total_casos

In [91]:
aux

,Código,Município,População 2010,2021,Casos/Hab 100M,Tx Crescimento dos anos 2021 e 2020
234,291955,Luís Eduardo Magalhães,60105,4329,7202.399902,1782.0
37,290320,Barreiras,137427,6642,4833.109863,648.0
134,291110,Formosa do Rio Preto,22528,861,3821.909912,86000.0
15,290140,Angical,14073,486,3453.419922,1844.0
274,292270,Nova Canaã,16713,565,3380.600098,87.0
268,292225,Muquém de São Francisco,10272,325,3163.939941,inf
54,290450,Brotas de Macaúbas,10717,325,3032.570068,693.0
408,293315,Várzea Nova,13073,344,2631.379883,514.0
158,291320,Ibotirama,25424,603,2371.770020,593.0
286,292335,Ourolândia,16425,365,2222.219971,75.0
